In [1]:
import threading
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from google.cloud import storage
from concurrent.futures import ThreadPoolExecutor, as_completed

BUCKET_NAME = 'chum_bucket_stuff'
SRC_DIR = Path("/Volumes/external_drive")

storage_client = storage.Client()
_bucket = storage_client.bucket("chum_bucket_stuff")

In [2]:
df = pd.read_csv("/Users/milosz.bertman/Projects/milfusion/datasets/galleries_dataset_with_dupes.csv")
df['gallery_info_path'] = SRC_DIR / df.gallery_category / df.gallery_name / "gallery_info.txt"
df = df.sort_values(by=["gallery_category", "gallery_name"])
df = df.reset_index(drop=True)

In [3]:
gallery_files = df.gallery_info_path.tolist()
gallery_files[0].as_posix()

'/Volumes/external_drive/amateur/18yearold-girls-lie-back-for-a-cumshot-after-an-intense-3some-fuck-38196807/gallery_info.txt'

In [4]:
def upload_to_gcp(source_file_name, destination_blob_name):
    blob = _bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)


threads = []
for file_path in tqdm(gallery_files, total=len(gallery_files), desc="Submitting Threads"):
    file_path = file_path.as_posix()
    destination_blob_name = file_path.replace(SRC_DIR.as_posix(), "pics")
    thread = threading.Thread(
        target=upload_to_gcp,
        args=(file_path, destination_blob_name)
    )
    threads.append(thread)
    thread.start()

# Use tqdm to show progress bar and wait for threads to complete
for thread in tqdm(threads, total=len(threads), desc="Uploading files"):
    thread.join()

Uploading files: 100%|██████████| 82588/82588 [00:00<00:00, 98210.21it/s] 
